In [ ]:
import pandas as pd
from parsing_multile_excels.calendat_utils import parse_time, parse_dates
from ical_element import Subject
from icalendar import Calendar

In [ ]:
df=pd.read_excel('complete.xlsx', index_col=0)
CURRENT_YEAR = 2022

In [ ]:
# Парсим Даты
df = df.groupby(['SubjectName', 'Place', 'Time', 'Teacher'])['Date'].agg(Dates = lambda x: x).reset_index()
df['Dates'] = df['Dates'].apply(lambda x: parse_dates(x))
df = df.explode('Dates')
df[['DateStart', 'DateEnd', 'RepeatTime', 'RepeatType']] = pd.DataFrame(df.Dates.tolist(), index=df.index)
df.head()

In [ ]:
# Парсим время
df[['TimeStart', 'TimeEnd']] = pd.DataFrame(df['Time'].apply(parse_time).tolist(), index=df.index)
df = df.drop(['Dates', 'Time'], axis=1)

In [ ]:
# Убираем все траектории (Ангийский и тд)
# df = df[df['SubjectName'].str.match('((^(?!Траектория).*)(^(?!Русский язык).*))|(Траектория 3 .*)')]

In [ ]:
# Создаем календарь
cal = Calendar()
cal.add('prodid', '-//Karpovich//Karpovich Alex//EN')

In [ ]:
# Добавляем туда события
for idx, item in df.iterrows():
    s = Subject.from_series(item)
    e = s.to_event()
    cal.add_component(e)

In [ ]:
# Сохраняем календарь. Ура
with open('autumn_2022.ics', 'wb') as f:
    data = cal.to_ical()
    f.write(data)